In [1]:
import os
import sys
import re
import time
import yaml
import shutil
import argparse
import tarfile
import tempfile
import random
import subprocess
import copy

import numpy as np
import pandas as pd

import torch

import boda
from boda.generator.parameters import BasicParameters
from boda.common import utils
from boda.common.utils import set_best, save_model, unpack_artifact, model_fn

import hypertune

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
test_path = 'gs://syrgoth/boda_library_design_202112/pickles/sg__hepg2__fsp__track_001__round_5__20211207_194303__533151.pt'
model_paths = [
    'gs://syrgoth/aip_ui_test/model_artifacts__20211113_021200__287348.tar.gz',
    'gs://syrgoth/aip_ui_test/model_artifacts__20211112_154705__841131.tar.gz',
    'gs://syrgoth/aip_ui_test/model_artifacts__20211113_205602__613479.tar.gz'
]

model_paths = ['gs://syrgoth/aip_ui_test/model_artifacts__20211113_021200__287348.tar.gz',
               'gs://syrgoth/aip_ui_test/model_artifacts__20211110_194934__672830.tar.gz',
               'gs://syrgoth/aip_ui_test/model_artifacts__20211119_011437__338420.tar.gz']

In [7]:
all_pickle_paths = !gsutil ls $'gs://syrgoth/boda_library_design_202112/pickles'

In [9]:
all_pickle_paths = [ path for path in all_pickle_paths if 'sg__' in path ]

In [3]:
def load_stash(filepath):
    if 'gs://' == filepath[:5]:
        with tempfile.TemporaryDirectory() as tmpdir:
            temp_loc = os.path.join( tmpdir, os.path.basename(filepath) )
            subprocess.check_call(
                ['gsutil', 'cp', filepath, temp_loc]
            )
            stash = torch.load(temp_loc)
    else:
        stash = torch.load(filepath)
    return stash

In [16]:
left_flank = boda.common.utils.dna2tensor( 
    boda.common.constants.MPRA_UPSTREAM[-200:] 
).unsqueeze(0)

right_flank= boda.common.utils.dna2tensor( 
    boda.common.constants.MPRA_DOWNSTREAM[:200] 
).unsqueeze(0)


def run_model(model, x):
    with torch.no_grad():
        bsz = x.shape[0]
        expander = [bsz, -1, -1]
        passthrough = BasicParameters(
            torch.randn_like(x), 
            left_flank = left_flank.expand(*expander), 
            right_flank = right_flank.expand(*expander)
        )
        passthrough.to(model.device)

        return model( passthrough(x.to(model.device)) ).cpu()

def load_stash_to_df(filepath, *model_paths, round_override=False):
    models = []
    if len(model_paths) > 0:
        print(f'loading {len(model_paths)} model(s)')
        for i, model_path in enumerate(model_paths):
            with tempfile.TemporaryDirectory() as tmpdirname:
                unpack_artifact(model_path, tmpdirname)
                _hold = model_fn(os.path.join(tmpdirname,'artifacts'))
                _hold.cuda()
                _hold.eval()
                models.append( _hold )            
    
    user, cell, gen, track_id, round_id, date_time, rand_id = os.path.basename(filepath).split('__')
    track_id = int(track_id.split('_')[1])
    date, time = [int(x) for x in date_time.split('_')]
    round_id = int(round_id.split('_')[1])
    rand_id = int(rand_id.split('.')[0])
    
    stash = load_stash(filepath)
    
    seq_list = []
    round_list = []
    
    model_results = { i: [] for i in range(len(models)) }
    
    for idx, proposal_round in enumerate(stash['proposals']):
        tensors = proposal_round['proposals']
        
        for j, model in enumerate(models):
            model_results[j].append( run_model(model, tensors) )
        
        seqs = list( utils.batch2list(tensors) )
        
        seq_list += seqs
        if round_override:
            round_list += [round_id] * len(seqs)
        else:
            round_list += [idx] * len(seqs)
        
    line_count = len(seq_list)
    
    data_dict =  {
        'sequence': seq_list,
        'penalty_round': round_list,
        'user': [ user ] * line_count,
        'generator':[ gen ] * line_count,
        'cell':[ cell ] * line_count,
        'track':[ track_id ] * line_count,
        'date':[ date ] * line_count,
        'time':[ time ] * line_count,
        'random_tag':[ rand_id ] * line_count,
    }

    relist_preds = []
    for i in range(len(models)):
        pred_set = model_results[i]
        cat_preds= torch.cat(pred_set, dim=0)
        n_preds, n_feats = cat_preds.shape
        relist_preds.append(cat_preds)
    avg_pred = torch.stack(relist_preds, dim=0).mean(0)
    for j in range(n_feats):
        data_dict[f'preds_cell_{j}'] = avg_pred[:,j].numpy()
    
    data = pd.DataFrame( data_dict )
    
    return data

In [5]:
my_data = load_stash_to_df(test_path, *model_paths)

loading 3 model(s)


archive unpacked in /tmp/tmpugtzdzt0


Loaded model from 20211113_021200 in eval mode


archive unpacked in /tmp/tmpz1l86koe


Loaded model from 20211110_194934 in eval mode


archive unpacked in /tmp/tmpk9ozuxu_


Loaded model from 20211119_011437 in eval mode


In [6]:
my_data

,sequence,penalty_round,user,generator,cell,track,date,time,random_tag,preds_cell_0,preds_cell_1,preds_cell_2
0,GATCTTGGACAGTGTTCAGAGCGCAGTAGTTATCCCGCTCTCCGAA...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.215762,6.250153,-0.540798
1,GGTGCAATTCCGTCGGGGGCCGTCGTGAGAATAGCTTGTATGCGGG...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.323467,6.939346,-0.773942
2,CTTCCCAAAGGCGTGAAGCAAGTGCATTGATTTAGGGTTGACGCAT...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.488120,4.720260,-0.777049
3,GGGTGCCCGCTGACCTGGTTAATGATTAACCAGGGGATTCTCAGCA...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.344566,11.012416,-0.617590
4,GGCTGGTTAATCATTAACCAGAAGACTCGCGCAACGGTCCAAAGTC...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.398559,8.773662,-0.538248
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,CAATCCCAAACATTAGTGATATGGCTCAGGTAGGTGACAGGAATAT...,5,sg,fsp,hepg2,1,20211207,194303,533151,0.834062,4.443956,0.797322
4496,TGCTAGCGCGGTTGAAAGATTGTAACCCAGATTTTCATGTTGGTGC...,5,sg,fsp,hepg2,1,20211207,194303,533151,0.062626,4.028974,1.139594
4497,CGTTACGACATGAACTGGCAATGCGTTTACCTCAGTAGTATAAGTT...,5,sg,fsp,hepg2,1,20211207,194303,533151,0.107993,2.329361,0.771194
4498,CATTACGGGTGCCGGGAAGGTGCGTGTGAATCCAGGGCATACAGCA...,5,sg,fsp,hepg2,1,20211207,194303,533151,0.052138,1.782599,0.179298


In [10]:
all_pickle_paths = !gsutil ls $'gs://syrgoth/boda_library_design_202112/pickles'
all_pickle_paths = [ path for path in all_pickle_paths if 'sg__' in path ]

sg_data = pd.concat([ load_stash_to_df(path, *model_paths) for path in all_pickle_paths ]).reset_index(drop=True)

loading 3 model(s)


archive unpacked in /tmp/tmpjeq8wszb


Loaded model from 20211113_021200 in eval mode


archive unpacked in /tmp/tmpihqvxuj_


Loaded model from 20211110_194934 in eval mode


...

Loaded model from 20211113_021200 in eval mode


archive unpacked in /tmp/tmp4crvr40d


Loaded model from 20211110_194934 in eval mode


archive unpacked in /tmp/tmptcyx7cfi


Loaded model from 20211119_011437 in eval mode


In [28]:
sg_data

In [22]:
all_pickle_paths = !gsutil ls $'gs://syrgoth/boda_library_design_202112/pickles'
all_pickle_paths = [ path for path in all_pickle_paths if 'rc__' in path ]

rc_data = pd.concat([ load_stash_to_df(path, *model_paths, round_override=True) for path in all_pickle_paths ]).reset_index(drop=True)

loading 3 model(s)


archive unpacked in /tmp/tmp3eu3e7yw


Loaded model from 20211113_021200 in eval mode


archive unpacked in /tmp/tmpkakpjruc


Loaded model from 20211110_194934 in eval mode


...

Loaded model from 20211113_021200 in eval mode


archive unpacked in /tmp/tmp8a_5hzdh


Loaded model from 20211110_194934 in eval mode


archive unpacked in /tmp/tmpf853c8li


Loaded model from 20211119_011437 in eval mode


In [26]:
rc_data

,sequence,penalty_round,user,generator,cell,track,date,time,random_tag,preds_cell_0,preds_cell_1,preds_cell_2
0,CATCTCACCTGTAAATCTCAATCAGTGCTTCGACAGATAACGGTCA...,0,rc,al,hepg2,2,20211206,25443,72806280,-1.299868,5.195824,-1.572185
1,TTAGCTCATCCTCTCCGTAGGGCGCACCTGCACAATCTGATGCTAG...,0,rc,al,hepg2,2,20211206,25443,72806280,-1.417148,5.089907,-1.658575
2,GCTCGATGCCAACGAGCCTATGACTAACATTCGATGCATACAGGTG...,0,rc,al,hepg2,2,20211206,25443,72806280,-1.393966,4.574755,-1.673908
3,GGCGCGACTCATTGTCATTGCCCAGTGCCATTCTATGTCAATATTT...,0,rc,al,hepg2,2,20211206,25443,72806280,-0.964659,3.159708,-1.039740
4,GCTCGCGCTGCGCACCTGTGCAAACAACCTTTGCCCGCCGAAGGCT...,0,rc,al,hepg2,2,20211206,25443,72806280,-1.539611,3.938985,-1.419131
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,CCCTTGGTCCACGACGAGTCAAACCATCTTTCGGTACCCTGGCATC...,0,rc,al,sknsh,7,20211207,212124,46481152,-1.001719,-1.032117,4.660513
14996,TAGAAGGAAATTACTAAGATGCGAGTGGTACCAGAGCTAACTTAAG...,0,rc,al,sknsh,7,20211207,212124,46481152,-0.452778,-0.416581,4.058228
14997,CCTGGCACCGAGTGATGGGCATACGCTCCATCATACACGGAAGGGC...,0,rc,al,sknsh,7,20211207,212124,46481152,-1.136951,-1.119118,5.341090
14998,GCTCGAATCTGGCCCGGATGCTGATTTAAGTATGAGTCAAGGAATC...,0,rc,al,sknsh,7,20211207,212124,46481152,-0.705151,-0.815807,4.297011


In [29]:
all_data = pd.concat([sg_data,rc_data]).reset_index(drop=True)

In [30]:
all_data

,sequence,penalty_round,user,generator,cell,track,date,time,random_tag,preds_cell_0,preds_cell_1,preds_cell_2
0,GATCTTGGACAGTGTTCAGAGCGCAGTAGTTATCCCGCTCTCCGAA...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.215762,6.250153,-0.540798
1,GGTGCAATTCCGTCGGGGGCCGTCGTGAGAATAGCTTGTATGCGGG...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.323467,6.939346,-0.773942
2,CTTCCCAAAGGCGTGAAGCAAGTGCATTGATTTAGGGTTGACGCAT...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.488120,4.720260,-0.777049
3,GGGTGCCCGCTGACCTGGTTAATGATTAACCAGGGGATTCTCAGCA...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.344566,11.012416,-0.617590
4,GGCTGGTTAATCATTAACCAGAAGACTCGCGCAACGGTCCAAAGTC...,0,sg,fsp,hepg2,1,20211207,194303,533151,-0.398559,8.773662,-0.538248
...,...,...,...,...,...,...,...,...,...,...,...,...
195784,CCCTTGGTCCACGACGAGTCAAACCATCTTTCGGTACCCTGGCATC...,0,rc,al,sknsh,7,20211207,212124,46481152,-1.001719,-1.032117,4.660513
195785,TAGAAGGAAATTACTAAGATGCGAGTGGTACCAGAGCTAACTTAAG...,0,rc,al,sknsh,7,20211207,212124,46481152,-0.452778,-0.416581,4.058228
195786,CCTGGCACCGAGTGATGGGCATACGCTCCATCATACACGGAAGGGC...,0,rc,al,sknsh,7,20211207,212124,46481152,-1.136951,-1.119118,5.341090
195787,GCTCGAATCTGGCCCGGATGCTGATTTAAGTATGAGTCAAGGAATC...,0,rc,al,sknsh,7,20211207,212124,46481152,-0.705151,-0.815807,4.297011
